In [1]:
#ライブラリのインポート 
import librosa
import os
import matplotlib.pyplot as plt
import numpy as np
import IPython.display
import pandas as pd
import statistics as sta
import torch

print(librosa.__version__)
print(torch.__version__)


0.10.2.post1
2.2.2


In [3]:
# # # 変数定義
sr = 44100 #サンプリング周波数

# #ファイルパスの指定
# origin_data = "data/origin_data/origindata_20240619"

# #学習用データの読み込み
# audio_dir = origin_data
# subFolders = [f for f in os.listdir(audio_dir) if os.path.isdir(os.path.join(audio_dir, f))]
# subFolders = sorted(subFolders) #昇順に並び替え

# #ファイル読み込み
# # originDatasに音声データを格納していく
# originDatas = []
# for subFolder in subFolders:
#     filePath = audio_dir+"/"+subFolder+"/"+subFolder+"_Tr1.WAV"
#     originData, sr = librosa.load(filePath, sr = sr)
#     originDatas.append(originData)

# #1データから30試行をトリミングを22パターン分行い，soundDataset_new_newに格納する
# # soundDataset_new_newの構造：[パターン(10)][試行(50)][サンプリングデータ(66150)]

# flag_amp = 0.1 #各試行の合図を検知する基準振幅
# trimSkip = int(sr*0.4)
# trimTime = int(sr*1.5)#1試行あたりのデータ長
# dataNum = 50
# trial = 0
# soundDataset_new =([])

# while trial <len(originDatas):
#     index = 0
#     trimDatas = ([]) #1データ(30試行)分のトリミングデータのリストを初期化
#     originData = np.array(originDatas[trial])
#     while index <len(originData):
#         if originData[index] >= flag_amp:
#             trimData = np.array(originData[index+trimSkip:index+trimTime+trimSkip]) #trim_dataにそのindexからindex+trimTimeのデータを格納する
#             trimDatas = np.append(trimDatas ,trimData, axis=0)  #trimDatasに追加する
#             index += trimTime
#         else:
#             index +=1
#         if len(trimDatas) >=dataNum*trimTime:
#             break
#     soundDataset_new = np.append(soundDataset_new,np.array(trimDatas),axis=0)
#     trial += 1

# soundDataset_new = soundDataset_new.reshape(len(originDatas),dataNum,trimTime)

# print(soundDataset_new.shape)

# import pickle
# f = open('soundDataset_20240619.pickle','wb')
# pickle.dump(soundDataset_new,f)

(21, 50, 66150)


In [4]:
import pickle
f = open('soundDataset_20240619.pickle','rb')
soundDataset_new = pickle.load(f)

In [7]:
#正解ラベルリストの作成

#object層
objectLabel = [
    "Y-shirt", 
    "Jeans", 
    "Sweatshirt", 

    "Blanket", 
    "bedquilt", 
    "Pillow", 
    
    "Thick Book", 
    "Thin Book", 
    "Cardboard", 
    
    "Mousepad", 
    "Chair", 
    "Sofa", 
    
    "Metal Desk", 
    "Laptop", 
    "AlumiRack", 
    
    "WoodDesk", 
    "WoodShelf", 
    "Floor", 
    
    "PlaCase", 
    "PlaContainer", 
    "PlaShelf"
]

#material層
matLabel = [
    "Clothing",
    "Clothing", 
    "Paper", 
    "Memory Foam", 
    "Metal", 
    "Wood", 
    "Plastic"
]

matNum = [0,0,1,2,3,4,5]

#soft-hard層
shLabel = [
    "soft",
    "hard",
]

shNum = [0,1]





In [5]:
# import soundfile as sf

# # wavファイルへの変換
# def MakeTrimData(target_num,trial):
#     # 既存のサンプル数66150の配列
#     audio_data = soundDataset_new[target_num][trial]

#     # WAVファイルに書き込む
#     output_file = 'data/trim_data/trim-data_'+str(target_num)+'_'+str(trial)+'.wav'
#     sample_rate = 44100  # サンプルレート（例: 44100 Hz）
#     sf.write(output_file, audio_data, sample_rate)

#     print(f"WAVファイル '{output_file}' に書き込みました。")

# # wavファイルへの変換実行
# for target_num in range(len(soundDataset_new)):
#     for trial in range(len(soundDataset_new[0])):
#         MakeTrimData(target_num,trial)

WAVファイル 'data/trim_data/trim-data_0_0.wav' に書き込みました。
WAVファイル 'data/trim_data/trim-data_0_1.wav' に書き込みました。
WAVファイル 'data/trim_data/trim-data_0_2.wav' に書き込みました。
WAVファイル 'data/trim_data/trim-data_0_3.wav' に書き込みました。
WAVファイル 'data/trim_data/trim-data_0_4.wav' に書き込みました。
WAVファイル 'data/trim_data/trim-data_0_5.wav' に書き込みました。
WAVファイル 'data/trim_data/trim-data_0_6.wav' に書き込みました。
WAVファイル 'data/trim_data/trim-data_0_7.wav' に書き込みました。
WAVファイル 'data/trim_data/trim-data_0_8.wav' に書き込みました。
WAVファイル 'data/trim_data/trim-data_0_9.wav' に書き込みました。
WAVファイル 'data/trim_data/trim-data_0_10.wav' に書き込みました。
WAVファイル 'data/trim_data/trim-data_0_11.wav' に書き込みました。
WAVファイル 'data/trim_data/trim-data_0_12.wav' に書き込みました。
WAVファイル 'data/trim_data/trim-data_0_13.wav' に書き込みました。
WAVファイル 'data/trim_data/trim-data_0_14.wav' に書き込みました。
WAVファイル 'data/trim_data/trim-data_0_15.wav' に書き込みました。
WAVファイル 'data/trim_data/trim-data_0_16.wav' に書き込みました。
WAVファイル 'data/trim_data/trim-data_0_17.wav' に書き込みました。
WAVファイル 'data/trim_data/trim-data_0_18

In [8]:
import pandas as pd
# num_feature次元のMFCCのデータフレームを作成

# ファイル名の生成と追加
filenames = []
for target_num in range(21):
    for trial in range(50):
        filenames.append(f'trim-data_{target_num}_{trial}.wav')

Dataset = pd.DataFrame()

Dataset['filename'] = filenames
Dataset['objectNum'] = [i // soundDataset_new.shape[1] for i in range(1050)]
Dataset['matNum'] = np.repeat(matNum, 150)[:1050]
Dataset['shNum'] = np.repeat(shNum, 600)[:1050]

# データセットに'objectLabel'の列を追加
Dataset['object'] = np.repeat(objectLabel, 50)[:1050]

# # データセットに'matLabel'の列を追加
Dataset['mat'] = np.repeat(matLabel, 150)[:1050]

# # データセットに'SHLabel'の列を追加
Dataset['sh'] = np.repeat(shLabel, 600)[:1050]


# データフレームをCSVファイルに保存
csv_path = 'data/vib_dataset.csv'
Dataset.to_csv(csv_path, index=False)

# Display the DataFrame
Dataset

,filename,objectNum,matNum,shNum,object,mat,sh
0,trim-data_0_0.wav,0,0,0,Y-shirt,Clothing,soft
1,trim-data_0_1.wav,0,0,0,Y-shirt,Clothing,soft
2,trim-data_0_2.wav,0,0,0,Y-shirt,Clothing,soft
3,trim-data_0_3.wav,0,0,0,Y-shirt,Clothing,soft
4,trim-data_0_4.wav,0,0,0,Y-shirt,Clothing,soft
...,...,...,...,...,...,...,...
1045,trim-data_20_45.wav,20,5,1,PlaShelf,Plastic,hard
1046,trim-data_20_46.wav,20,5,1,PlaShelf,Plastic,hard
1047,trim-data_20_47.wav,20,5,1,PlaShelf,Plastic,hard
1048,trim-data_20_48.wav,20,5,1,PlaShelf,Plastic,hard
